# 2D Reactor with Heterogenous Chemistry
## No Vertical Velocity
### Do not run without stopping quickly or it will take up too much memory

In [1]:
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt
import pylab
import matplotlib.gridspec as gridspec
import cantera as ct
from scikits.odes import dae
%matplotlib inline
print("Running Cantera Version: " + str(ct.__version__))


Running Cantera Version: 2.5.1


## Reactor Conditions

In [2]:
#--------------------------------------------------------
# composition of the inlet premixed gas for the hydrogen/air case
comp = 'H2:0.6, O2:0.8, N2:0.2'
Tinlet = 300.0  # inlet temperature, K
Tsurf = 900.0  # surface temperature, K
p0 = ct.one_atm # pressure, Pa
# pipe parameters
L = 1. # m
H = 0.1 # m
# load H2-O2 gas
gas = ct.Solution('ptcombust.yaml', 'gas', transport_model='Mix')
gas.TPX = Tinlet, p0, comp # set initial gas conditions
# load Pt surf
surf = ct.Interface('ptcombust.yaml', 'Pt_surf', [gas])
surf.TP = Tsurf, p0
# get to steady state conditions
surf.advance_coverages(100.0)

Nk = gas.n_species
Ns = surf.n_species
Nz = 51
delz = H/(Nz-1)
W = gas.molecular_weights


## Residual DAE solver function

In [3]:
def residual(x, nvec, ndvec, result):
    vec = nvec[Ns:].reshape([Nz,4+Nk]) # from 1D array to 2D array for easier indexing
    dvec = ndvec[Ns:].reshape([Nz,4+Nk]) # from 1D array to 2D array for easier indexing
    """
    vec = [[u_0, rho_0, T_0, p_0, yk_0]
           [u_1, rho_1, T_1, p_1, yk_1]
           ...
           [u_Nz, rho_Nz, T_Nz, p_Nz, yk_Nz]
           [Z_0...Z_Ns]]
           
    dvec = [[dudx_0, drhodx_0, dTdx_0, dpdx_0, dykdx_0]
            ...
            [dudx_Nz, drhodx_Nz, dTdx_Nz, dpdx_Nz, dykdx_Nz]
            [dZdx_0...dZdx_Ns]]
    """

    #--------------------------------------------------------
    # loop through values in z direction
    for j in range(0,Nz):
        # define variables
        u = vec[j,0]
        udown = vec[j-1,0]
        rho = vec[j,1]
        rhodown = vec[j-1,1]
        T = vec[j,2]
        Tdown = vec[j-1,2]
        p = vec[j,3]
        pdown = vec[j-1,3]
        yk = vec[j,4:4+Nk] # yk[j,k] where j is index of Z and k is species
        ykdown = vec[j-1,4:4+Nk]
        Zk = nvec[:Ns]
        
        if j != Nz-1:
            uup = vec[j+1,0]
            rhoup = vec[j+1,1]
            Tup = vec[j+1,2]
            pup = vec[j+1,3]
            ykup = vec[j+1,4:4+Nk]

        # define x derivatives
        dudx = dvec[j,0]
        drhodx = dvec[j,1]
        dTdx = dvec[j,2]
        dpdx = dvec[j,3]
        dykdx = dvec[j,4:4+Nk]
        dZkdx = ndvec[:Ns]
        gas.set_unnormalized_mass_fractions(yk)
        gas.TP = Tinlet, p
        
        # gas
        W = gas.molecular_weights # kg/kmol
        
        wdot = gas.net_production_rates
        h = gas.enthalpy_mass
        mu = gas.viscosity
        Dk = gas.mix_diff_coeffs_mass
        Wmean = gas.mean_molecular_weight
        
        
        # floor layer
        if j == 0:
            ## temporary variables for each integration, surface
            # surface
            surf.set_unnormalized_coverages(Zk)
            surf.TP = Tsurf, p
            coverages = surf.coverages
            sdotg = surf.get_net_production_rates(1) # kmol/m^3/s
            sdots = surf.get_net_production_rates(0) # kmol/m^2/s
            
            
            # conservation of mass
            result[0] = np.sum(sdotg*W)
            # conservation of momentum x
            result[1] = (dpdx - mu*(2.*uup)/delz**2.)
            # conservation of energy
            result[2] = T - Tinlet
            # equation of state
            result[3] = rho - gas.density
            # conservation of species N-1
            result[4:4+Nk] = -rho*Dk*(ykup - yk)/delz - sdotg*W
            ####
            # Algebraic Constraints
            # replace nitrogen with sum of gas species mass fractions = 1
            result[4+Nk-1] = np.sum(yk) - 1.
            # species production
            result[4+Nk:4+Nk+Ns] = sdots
            # surface site fraction sum = 1
            index = np.argmax(coverages)
            result[4+Nk+index] = np.sum(coverages) - 1.

        # middle
        elif j > 0 and j < Nz-1:
            middlei = 4+Nk+Ns
            # conservation of mass
            result[middlei+(j-1)*middlei] = u*drhodx + rho*dudx
            # conservation of momentum x
            result[middlei+(j-1)*middlei+1] = (rho*u*dudx 
                                               + dpdx 
                                               - mu*((uup-2.*u+udown)/delz**2.))
            # conservation of energy
            result[middlei+(j-1)*middlei+2] = T - Tinlet
            # eqn of state
            result[middlei+(j-1)*middlei+3] = rho - gas.density
            # conservation of gas species N-1
            result[middlei+(j-1)*middlei+4:middlei+(j-1)*middlei+4+Nk] = (- rho*u*dykdx 
                                                                          - rho*Dk*(ykup-2.*yk+ykdown)/delz**2.)
            # conservation of gas species fractions
            index1 = middlei+(j-1)*middlei+4+Nk-1
            result[index1] = np.sum(yk) - 1.
        
            
        # ceiling layer (implicit boundary conditions)
        elif j == Nz-1:
            # new starting index for ceiling
            ceili = middlei+(Nz-2-1)*middlei+4+Nk
            # conservation of mass (no slip and dvdz = 0)
            result[ceili] = u
            # conservation of momentum, x 
            result[ceili+1] = (dpdx - mu*(2.*udown/delz**2.))
            # conservation of energy
            result[ceili+2] = T - Tinlet
            # equation of State
            result[ceili+3] = rho - gas.density
            # conservation of species N-1
            result[ceili+4:ceili+4+Nk] = yk - ykdown
            ## algebraic 
            # conservation of gas species fractions
            result[ceili+4+Nk-1] = np.sum(yk) - 1.
            

## Determine Initial Conditions

In [4]:
# initial conditions
u0 = 0.5 #m/s
rho0 = gas.density
indexes = [0,3,-1] # indexes of initial composition 
W0_avg = np.mean(gas.molecular_weights[indexes])
mu0 = gas.viscosity

# middle
vecmiddle0 = np.hstack((u0, rho0, Tinlet, p0, gas.Y))
vecmiddle_total0 = np.hstack((u0, rho0, Tinlet, p0, gas.Y))
for j in range(2, Nz-1):
    vecmiddle_total0 = np.vstack((vecmiddle_total0, vecmiddle0))
dvecmiddle_total0 = np.zeros((Nz-2,4+Nk))

# ceiling
pprime = -mu0*u0/delz**2.
vecceil0 = np.hstack((0, rho0, Tinlet, p0, gas.Y))
dvecceil0 = np.hstack((0, 0, 0, pprime, np.zeros(Nk)))

# surf
vecsurf0 = np.hstack((0, rho0, Tinlet, p0, gas.Y))
dvecurf0 = np.hstack((0, 0, 0, pprime, np.zeros(Nk)))

# Z
Zk0 = surf.coverages
vec0_part = np.vstack((vecsurf0, vecmiddle_total0, vecceil0)).reshape((1,Nz*(4+Nk)))[0]
vec0 = np.hstack((Zk0, vec0_part))

dvec0_part = np.vstack((dvecurf0, dvecmiddle_total0, dvecceil0)).reshape((1,Nz*(4+Nk)))[0]
dvec0 = np.hstack((np.zeros(Ns), dvec0_part))


## Call DAE Solver

In [ ]:
# get the index's of the algebraic constraint equations
algebraic_vars_idx = np.hstack((np.arange(5+Nk-1,5+Nk+Ns)))
for j in range(Nz):
    if j > 0 and j < Nz-1:
        middlei = 5+Nk+Ns
        algebraic_vars_idx = np.hstack((algebraic_vars_idx, np.arange(middlei+(j-1)*middlei+5+Nk-1,middlei+(j-1)*middlei+5+Nk+Ns)))
ceili = middlei+(Nz-2-1)*middlei+5+Nk+Ns
algebraic_vars_idx = np.hstack((algebraic_vars_idx, np.arange(ceili+5+Nk-1,ceili+5+Nk+Ns)))
algebraic_vars_idx = algebraic_vars_idx.tolist()

# solve DAE
solver = dae(
    'ida',
    residual, 
    atol=1e-8,  # absolute tolerance for solution
    rtol=1e-8,  # relative tolerance for solution
    algebraic_vars_idx=algebraic_vars_idx, 
    max_steps=5000,
    one_step_compute=True,
    old_api=False
)

# xaxis = []
# solution = []
# state = solver.init_step(0.0, vec0, dvec0)
# while state.values.t < 0.1:
#     print(state.values.t)
#     xaxis.append(state.values.t)
#     solution.append(state.values.y)
#     state = solver.step(0.01)

# xaxis = np.array(xaxis)
# solution = np.array(solution)

times = np.linspace(0,0.1,10)
solution = solver.solve(times, vec0, dvec0)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Exception ignored in: 'scikits.odes.sundials.ida._res'
Traceback (most recent call last):
  File "scikits/odes/sundials/ida.pyx", line 159, in scikits.odes.sundials.ida.IDA_WrapRhsFunction.evaluate
  File "<ipython-input-3-b9093d50aa14>", line 21, in residual
IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Exception ignored in: 'scikits.odes.sundials.ida._res'
Traceback (most recent call last):
  File "scikits/odes/sundials/ida.pyx", line 159, in scikits.odes.sundials.ida.IDA_WrapRhsFunction.evaluate
  File "<ipython-input-3-b9093d50aa14>", line 21, in residual
IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

...

Exception ignored in: 'scikits.odes.sundials.ida._res'
Traceback (most recent call last):
  File "scikits/odes/sundials/ida.pyx", line 159, in scikits.odes.sundials.ida.IDA_WrapRhsFunction.evaluate
  File "<ipython-input-3-b9093d50aa14>", line 21, in residual
IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Exception ignored in: 'scikits.odes.sundials.ida._res'
Traceback (most recent call last):
  File "scikits/odes/sundials/ida.pyx", line 159, in scikits.odes.sundials.ida.IDA_WrapRhsFunction.evaluate
  File "<ipython-input-3-b9093d50aa14>", line 21, in residual
IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Exception ignored in: 'scikits.odes.sundials.ida._res'
Traceback (most recent call last):
  File "scikits/odes/sundials/ida.pyx", line 159, in scikits.odes.sundials.ida.IDA_WrapRhsFunction.evaluate


In [ ]:
solution.message
